# Initial Data Building: 

In [9]:
import pandas as pd

In [10]:
injuries_df = pd.read_csv('data/injuries_2024.csv')
play_by_play_df = pd.read_csv('data/play_by_play_2024.csv')

/var/folders/b2/0jnp196543qfsgdbvzhy42pr0000gn/T/ipykernel_10400/1838363699.py:2: DtypeWarning: Columns (179,180,182,183,189,190,197,198,203,204,222,224,226,283,284) have mixed types. Specify dtype option on import or set low_memory=False.
  play_by_play_df = pd.read_csv('data/play_by_play_2024.csv')


In [13]:
print(f"Columns in injuries_2024.csv ({len(injuries_df.columns)} columns):")
print(injuries_df.columns.tolist())
print(f"\nColumns in play_by_play_2024.csv ({len(play_by_play_df.columns)} columns):")
print(play_by_play_df.columns.tolist())

Columns in injuries_2024.csv (16 columns):
['season', 'game_type', 'team', 'week', 'gsis_id', 'position', 'full_name', 'first_name', 'last_name', 'report_primary_injury', 'report_secondary_injury', 'report_status', 'practice_primary_injury', 'practice_secondary_injury', 'practice_status', 'date_modified']

Columns in play_by_play_2024.csv (372 columns):
['play_id', 'game_id', 'old_game_id', 'home_team', 'away_team', 'season_type', 'week', 'posteam', 'posteam_type', 'defteam', 'side_of_field', 'yardline_100', 'game_date', 'quarter_seconds_remaining', 'half_seconds_remaining', 'game_seconds_remaining', 'game_half', 'quarter_end', 'drive', 'sp', 'qtr', 'down', 'goal_to_go', 'time', 'yrdln', 'ydstogo', 'ydsnet', 'desc', 'play_type', 'yards_gained', 'shotgun', 'no_huddle', 'qb_dropback', 'qb_kneel', 'qb_spike', 'qb_scramble', 'pass_length', 'pass_location', 'air_yards', 'yards_after_catch', 'run_location', 'run_gap', 'field_goal_result', 'kick_distance', 'extra_point_result', 'two_point_con

In [17]:
print(f"Number of rows in play_by_play: {play_by_play_df.shape[0]}")
print(f"Number of rows in injuries: {injuries_df.shape[0]}")

Number of rows in play_by_play: 15913
Number of rows in injuries: 1617


In [14]:
common_columns = set(injuries_df.columns).intersection(play_by_play_df.columns)

if common_columns:
    print(f"Common columns between injuries_2024.csv and play_by_play_2024.csv ({len(common_columns)} columns):")
    print(common_columns)
else:
    print("There are no common columns between injuries_2024.csv and play_by_play_2024.csv.")

Common columns between injuries_2024.csv and play_by_play_2024.csv (2 columns):
{'week', 'season'}


In [15]:
player_id_columns = [col for col in play_by_play_df.columns if '_player_id' in col]

if player_id_columns:
    print(f"Columns containing '_player_id' in play_by_play_2024.csv ({len(player_id_columns)} columns):")
    print(player_id_columns)
else:
    print("No columns containing '_player_id' in play_by_play_2024.csv.")

Columns containing '_player_id' in play_by_play_2024.csv (43 columns):
['td_player_id', 'passer_player_id', 'receiver_player_id', 'rusher_player_id', 'lateral_receiver_player_id', 'lateral_rusher_player_id', 'lateral_sack_player_id', 'interception_player_id', 'lateral_interception_player_id', 'punt_returner_player_id', 'lateral_punt_returner_player_id', 'kickoff_returner_player_id', 'lateral_kickoff_returner_player_id', 'punter_player_id', 'kicker_player_id', 'own_kickoff_recovery_player_id', 'blocked_player_id', 'tackle_for_loss_1_player_id', 'tackle_for_loss_2_player_id', 'qb_hit_1_player_id', 'qb_hit_2_player_id', 'forced_fumble_player_1_player_id', 'forced_fumble_player_2_player_id', 'solo_tackle_1_player_id', 'solo_tackle_2_player_id', 'assist_tackle_1_player_id', 'assist_tackle_2_player_id', 'assist_tackle_3_player_id', 'assist_tackle_4_player_id', 'tackle_with_assist_1_player_id', 'tackle_with_assist_2_player_id', 'pass_defense_1_player_id', 'pass_defense_2_player_id', 'fumbled_

In [20]:
total_matches = 0 

for col in player_id_columns:
    
    matching_rows = play_by_play_df[play_by_play_df[col].isin(injuries_df['gsis_id'])]
    
    
    num_matches = matching_rows.shape[0]
    
    if num_matches > 0:
        print(f"Matches found in column '{col}': {num_matches} rows")
        print(matching_rows[[col]])
        total_matches += num_matches  
    else:
        print(f"No matches found in column '{col}'.")


print(f"Total number of matched rows across all columns: {total_matches}")

Matches found in column 'td_player_id': 255 rows
      td_player_id
72      00-0034857
87      00-0033555
104     00-0037261
128     00-0034857
184     00-0039894
...            ...
15521   00-0032211
15528   00-0029892
15558   00-0036411
15594   00-0037545
15626   00-0033906

[255 rows x 1 columns]
Matches found in column 'passer_player_id': 1779 rows
      passer_player_id
3           00-0035228
4           00-0035228
8           00-0035228
9           00-0035228
13          00-0035228
...                ...
15334       00-0038579
15345       00-0038579
15346       00-0038579
15347       00-0038579
15348       00-0038579

[1779 rows x 1 columns]
Matches found in column 'receiver_player_id': 3379 rows
      receiver_player_id
8             00-0035500
9             00-0039849
19            00-0033555
24            00-0037744
27            00-0035500
...                  ...
15866         00-0030061
15871         00-0030061
15875         00-0033591
15882         00-0036550
15884        

1. check if data that matches with gsis_id 

# Select Data: 

# Clean Data: 

# Construct Data: 

# Integrate Data: 

# Format Data: 